In [2]:
import os,random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
dictOfStrings={"%3A":":","%2F":"/"}
import warnings
warnings.filterwarnings("ignore")

In [3]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\rkumarrakeshkumar\.wdm\drivers\chromedriver\win32\105.0.5195.52]


In [1]:
zips = [12550] #,41501,46733,30501,19803,19707

In [4]:
GlobalData=pd.DataFrame()
counter=0
for zipcode in zips:
    print("ZipCode:- ",zipcode)
    print("Counter:- ",counter)
    browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
    list1=[]
    for i in browser.find_elements_by_class_name("css-chan6m"):
        list1.append(i.text)
    noOfPages=int(list1[-4][-2:])
    customerList=[]
    SubMenu=[]
    menuDesc=[]
    Menu=[]
    custDict={}
    FinalList=[]
    restType=[]
    RestaurantType=[]
    LocalData=pd.DataFrame()
    LocalDataPage0=pd.DataFrame()
    print("No of Pages:",noOfPages)
    print("********************************************")
    for page in range(noOfPages):
        if(page==0):
#             print("--------------------------------------------Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
#                 print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])
                
                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))

                        
                    data.loc[0,"RestaurantType"]=RestaurantType
                    browser.find_elements_by_class_name("css-qyp8bo")
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])

                    
    
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
                
                    
                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
                    
            
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
                    
                    Menu["Key"]=1
    
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])

                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))

        else:
#             print("--------------------------------------------Else Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+str("0"*(5-len(str(zipcode)))+str(zipcode))+"&start="+str(page*10))
            browser.switch_to.window(browser.window_handles[0])
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
#                 print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])

                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))
                    data.loc[0,"RestaurantType"]=RestaurantType
                    

                    browser.find_elements_by_class_name("css-qyp8bo")
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])
                
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
                

                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
    
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
#                     print("SubMenu",SubMenu)
                    Menu["Key"]=1
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])
                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))
                    LocalData=LocalData.dropna(subset=['ZipCode'])
                    

#     print("Append Zipcode",counter)
    
    GlobalData=GlobalData.append(LocalData, ignore_index=True)
    GlobalData.to_csv(r'C:\Users\rkumarrakeshkumar\Desktop\AI_Guild_Capstone_Project\Globaldata.csv',index=False)
        
#     if (counter %  5 ==0) & (counter!=0):
#         print("****Writing Data****",counter)
#         GlobalData.to_excel(r"CustomerAcquisition/customerAcquisition_till_Zipcode_"+zipcode+"_"+str(counter+1)+"_Zipcodes"+".xlsx")
#         GlobalData=pd.DataFrame()
#     counter+=1


ZipCode:-  12550
Counter:-  0
No of Pages: 24
********************************************


In [5]:
GlobalData.shape

(7295, 7)

In [6]:
GlobalData.head()

,Customer,ZipCode,RestaurantType,Menu,MenuDescription,Price,CityZipcode
0,1 Blu Pointe,12550,"Seafood, Steakhouses, Wine Bars",Whole Maine Lobster Cocktail,"Bloody Mary Sauce, Lemon-Thyme Aioli",$48.00,12550
1,1 Blu Pointe,12550,"Seafood, Steakhouses, Wine Bars",Crab Cocktail,"Colossal Lump Crab Meat, Lemon-Thyme Aioli, Bl...",$48.00,12550
2,1 Blu Pointe,12550,"Seafood, Steakhouses, Wine Bars",Colossal Shrimp Cocktail,"Horseradish, Bloody Mary Sauce",$30.00,12550
3,1 Blu Pointe,12550,"Seafood, Steakhouses, Wine Bars",Littleneck Clams on the Half-Shell,"Riesling Mignonette, Bloody Mary Sauce",Dozen $16.00,12550
4,1 Blu Pointe,12550,"Seafood, Steakhouses, Wine Bars",Oysters on the Half-Shell,"Riesling Mignonette, Bloody Mary Sauce",Kumamoto $5.00,12550


In [ ]:
# 2:52 PM triggered the code

In [8]:
res_with_menu = GlobalData.Customer.unique().tolist()

In [ ]:
reviews

In [52]:
GlobalData.shape

(17044, 7)

In [53]:
GlobalData.head()

,Customer,ZipCode,RestaurantType,Menu,MenuDescription,Price,CityZipcode
0,1 Ikaros Restaurant Diner,12553,"Diners, Greek, American (Traditional)",French Onion Soup,"Served in a crock with home-made croutons, moz...",$8.00,12550
1,1 Ikaros Restaurant Diner,12553,"Diners, Greek, American (Traditional)",Spinach and Artichoke Dip,Creamy fresh spinach and artichokes baked with...,$10.25,12550
2,1 Ikaros Restaurant Diner,12553,"Diners, Greek, American (Traditional)",Shrimp Saganaki,"Jumbo shrimp, fresh tomatoes, onions, garlic, ...",$15.95,12550
3,1 Ikaros Restaurant Diner,12553,"Diners, Greek, American (Traditional)",Fresh Mozzarella and Basil,"Fresh mozzarella served with sliced tomatoes, ...",$13.00,12550
4,1 Ikaros Restaurant Diner,12553,"Diners, Greek, American (Traditional)",Nachos,"Crispy tri-color tortilla chips, topped with c...",$11.45,12550


In [54]:
GlobalData.groupby(['CityZipcode']).agg({'Customer':'nunique'})

,Customer
CityZipcode,
12550,95
30501,118
41501,7
46733,5


In [55]:
GlobalData.to_csv(r'C:\Users\rkumarrakeshkumar\Desktop\AI_Guild_Capstone_Project\All_Global_data.csv',index=False)

In [ ]:
# GlobalData=GlobalData.append(LocalData, ignore_index=True)

# GlobalData.to_excel(r"CustomerAcquisition/customerAcquisition_till_Zipcode_"+zipcode+"_"+str(counter+1)+"_Zipcodes"+".xlsx")

# missingZipcodes.to_excel(r"missingZipcodes_till_Zipcode_"+zipcode+"_"+str(counter+1)+"_Zipcodes"+".xlsx")


In [88]:
browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166")
rest=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
for i in rest:
    print(i.text.split(' ')[-1])

19


In [92]:
pages=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
for i in pages:
    print(i.text.split('\n')[-1][-3:])

 19


In [82]:
browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166")
rest=browser.find_elements_by_class_name("border-color--default__09f24__NPAKY")
for i in rest:
    print(i.text.split('\n'))

['']
['Yelp', 'For Businesses', 'Write a Review', 'Log In', 'Sign Up', 'Restaurants', 'Home Services', 'Auto Services', 'More']
['']
['']
['Yelp', 'For Businesses', 'Write a Review', 'Log In', 'Sign Up', 'Restaurants', 'Home Services', 'Auto Services', 'More']
['Yelp', 'For Businesses', 'Write a Review', 'Log In', 'Sign Up', 'Restaurants', 'Home Services', 'Auto Services', 'More']
['Yelp', 'For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['Yelp']
['Yelp']
['Yelp']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['']
['']
['']
['']
['']
['']
['For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['For Businesses', 'Write a Review', 'Log In', 'Sign Up']
['For Businesses']
['For Businesses']
['For Businesses']
['Write a Review']
['Write a Review']
['W

['Write a review', 'Add photo', 'Share', 'Save', 'Follow', 'From This Business', 'Find Your Fall', 'Fall drinks now available. Order at Starbucks today.', 'Menu', 'Website menu', 'Full menu', 'Review Highlights', '“Wondering if Lily still works here, because the customer service did not seem up to par as it was before.” in 8 reviews', '“your friendly baristas that keep me entertained from the real world of nefarious pee-smelling wanderers and 50 cent pan-handlers.” in 2 reviews', '“Oh I...I need the in-store and mobile order to go, When my energy is running low” in 2 reviews', 'Location & Hours', '1390 Market St', 'Fox Plaza', 'San Francisco, CA 94102', 'Larkin St & 9th St', 'Civic Center', 'Get directions', 'Mon', '5:00 AM - 7:30 PM', 'Closed now', 'Tue', '5:00 AM - 7:30 PM', 'Wed', '5:00 AM - 7:30 PM', 'Thu', '5:00 AM - 7:30 PM', 'Fri', '5:00 AM - 7:30 PM', 'Sat', '5:30 AM - 6:00 PM', 'Sun', '5:30 AM - 6:00 PM', 'Edit business info', 'Amenities and More', 'Health ScorePASS', 'Powered

['Write a review', 'Add photo', 'Share', 'Save', 'Follow', 'From This Business', 'Find Your Fall', 'Fall drinks now available. Order at Starbucks today.', 'Menu', 'Website menu', 'Full menu', 'Review Highlights', '“Wondering if Lily still works here, because the customer service did not seem up to par as it was before.” in 8 reviews', '“your friendly baristas that keep me entertained from the real world of nefarious pee-smelling wanderers and 50 cent pan-handlers.” in 2 reviews', '“Oh I...I need the in-store and mobile order to go, When my energy is running low” in 2 reviews', 'Location & Hours', '1390 Market St', 'Fox Plaza', 'San Francisco, CA 94102', 'Larkin St & 9th St', 'Civic Center', 'Get directions', 'Mon', '5:00 AM - 7:30 PM', 'Closed now', 'Tue', '5:00 AM - 7:30 PM', 'Wed', '5:00 AM - 7:30 PM', 'Thu', '5:00 AM - 7:30 PM', 'Fri', '5:00 AM - 7:30 PM', 'Sat', '5:30 AM - 6:00 PM', 'Sun', '5:30 AM - 6:00 PM', 'Edit business info', 'Amenities and More', 'Health ScorePASS', 'Powered

['Write a review', 'Add photo', 'Share', 'Save', 'Follow', 'From This Business', 'Find Your Fall', 'Fall drinks now available. Order at Starbucks today.', 'Menu', 'Website menu', 'Full menu', 'Review Highlights', '“Wondering if Lily still works here, because the customer service did not seem up to par as it was before.” in 8 reviews', '“your friendly baristas that keep me entertained from the real world of nefarious pee-smelling wanderers and 50 cent pan-handlers.” in 2 reviews', '“Oh I...I need the in-store and mobile order to go, When my energy is running low” in 2 reviews', 'Location & Hours', '1390 Market St', 'Fox Plaza', 'San Francisco, CA 94102', 'Larkin St & 9th St', 'Civic Center', 'Get directions', 'Mon', '5:00 AM - 7:30 PM', 'Closed now', 'Tue', '5:00 AM - 7:30 PM', 'Wed', '5:00 AM - 7:30 PM', 'Thu', '5:00 AM - 7:30 PM', 'Fri', '5:00 AM - 7:30 PM', 'Sat', '5:30 AM - 6:00 PM', 'Sun', '5:30 AM - 6:00 PM', 'Edit business info', 'Amenities and More', 'Health ScorePASS', 'Powered

['Write a review', 'Add photo', 'Share', 'Save', 'Follow', 'From This Business', 'Find Your Fall', 'Fall drinks now available. Order at Starbucks today.', 'Menu', 'Website menu', 'Full menu', 'Review Highlights', '“Wondering if Lily still works here, because the customer service did not seem up to par as it was before.” in 8 reviews', '“your friendly baristas that keep me entertained from the real world of nefarious pee-smelling wanderers and 50 cent pan-handlers.” in 2 reviews', '“Oh I...I need the in-store and mobile order to go, When my energy is running low” in 2 reviews', 'Location & Hours', '1390 Market St', 'Fox Plaza', 'San Francisco, CA 94102', 'Larkin St & 9th St', 'Civic Center', 'Get directions', 'Mon', '5:00 AM - 7:30 PM', 'Closed now', 'Tue', '5:00 AM - 7:30 PM', 'Wed', '5:00 AM - 7:30 PM', 'Thu', '5:00 AM - 7:30 PM', 'Fri', '5:00 AM - 7:30 PM', 'Sat', '5:30 AM - 6:00 PM', 'Sun', '5:30 AM - 6:00 PM', 'Edit business info', 'Amenities and More', 'Health ScorePASS', 'Powered

...

['0', '8', '4']
['0', '8', '4']
['0']
['0']
['8']
['8']
['4']
['4']
['']
['']
['9/6/2022']
['9/6/2022']
['']
['']
['']
['']
['']
['']
['']
['']
['9/6/2022']
['I have been coming to this store daily for over 5 years. Staff is very friendly and always greet me by my name. The coffee is very consistent in taste and quality. Hoping the management can keep the same staff for a long period of time.']
['Useful', 'Funny', 'Cool']
['Useful', 'Funny', 'Cool']
['Useful']
['Useful']
['']
['Useful']
['Useful']
['Funny']
['Funny']
['']
['Funny']
['Funny']
['Cool']
['Cool']
['']
['Cool']
['Cool']
['Colleen C.', 'Beverly Hills, CA', '7', '7', '11', '8/5/2022', '1 photo', "Had my coffee with Starbucks yesterday and I got to say I missed it! I missed the smell of the stores when you come in, the sound of the grinding and dripping coffee beans, and the greetings from the staff. I ordered their Café Latte with double shots and it help me kicked off my day. I'm thinking of having a coffee again with Starbu

['Kenneth W.', 'Berkeley, CA', '489', '17', '8']
['Kenneth W.', 'Berkeley, CA', '489', '17', '8']
['Kenneth W.', 'Berkeley, CA', '489', '17', '8']
['Kenneth W.', 'Berkeley, CA', '489', '17', '8']
['']
['']
['Kenneth W.', 'Berkeley, CA', '489', '17', '8']
['Kenneth W.', 'Berkeley, CA']
['Berkeley, CA']
['Berkeley, CA']
['489', '17', '8']
['489', '17', '8']
['489', '17', '8']
['489']
['489']
['17']
['17']
['8']
['8']
['']
['']
['3/10/2022']
['3/10/2022']
['']
['']
['']
['']
['']
['']
['']
['']
['3/10/2022']
['1 photo']
['1 photo']
['']
["Keily provided the worst customer service I have ever experienced at a Starbucks. After taking the order, one drink was missing and I was told I didn't order 4 drinks. Keily then expressed a level of attitude that is unacceptable and when called out as to why she said I don't have an attitude, you didn't order. So I order the drink again and she confirms mangel Stanton fruit no lemonade. I get the drink and the barista said it has lemonade. So not only d

['Useful']
['Useful']
['Funny 1']
['Funny 1']
['']
['Funny 1']
['Funny 1']
['Cool']
['Cool']
['']
['Cool']
['Cool']
['Jay S.', 'Tampa, FL', '194', '44', '11', '12/5/2021', 'Food was overpriced and mediocre. Would not recommend. Service was ok. We went on a Friday night and it was fairly busy for this restaurant.', 'Useful', 'Funny 1', 'Cool']
['Jay S.', 'Tampa, FL', '194', '44', '11', '12/5/2021', 'Food was overpriced and mediocre. Would not recommend. Service was ok. We went on a Friday night and it was fairly busy for this restaurant.', 'Useful', 'Funny 1', 'Cool']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['']
['']
['Jay S.', 'Tampa, FL', '194', '44', '11']
['Jay S.', 'Tampa, FL']
['Tampa, FL']
['Tampa, FL']
['194', '44', '11']
['194', '44', '11']
['194', '44', '11']
['194']
['194']
['44']
['44']
['11']
['11']
['

['Desserts,']
['Coffee & Tea,']
['Delis']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['']
['']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee', '50', '$', 'Coffee & Tea']
['Peet’s Coffee']
['Peet’s Coffee']
['50']
['50']
['']
['']
['']
['']
['']
['']
['']
['']
['50']
['$']
['$']
['Coffee & Tea']
['Coffee & Tea']
['Coffee & Tea']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['']
['']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz Coffee', '203', '$', 'Coffee & Tea, Breakfast & Brunch']
['Philz 

['What forms of payment are accepted?', 'Starbucks accepts credit cards.']
['How is Starbucks rated?', 'Starbucks has 3 stars.']
['What days are Starbucks open?', 'Starbucks is open Mon, Tue, Wed, Thu, Fri, Sat, Sun.']
['About', 'About Yelp', 'Careers', 'Press', 'Investor Relations', 'Trust & Safety', 'Content Guidelines', 'Accessibility Statement', 'Terms of Service', 'Privacy Policy', 'Ad Choices', 'Discover', 'Yelp Project Cost Guides', 'Collections', 'Talk', 'Events', 'The Local Yelp', 'Yelp Blog', 'Support', 'Yelp Mobile', 'Developers', 'RSS', 'Yelp for Business', 'Claim your Business Page', 'Advertise on Yelp', 'Yelp for Restaurant Owners', 'Table Management', 'Business Success Stories', 'Business Support', 'Yelp Blog for Business', 'Languages', 'English', 'Countries', 'United States', 'Copyright © 2004–2022 Yelp Inc. Yelp, , and related marks are registered trademarks of Yelp.']
['About', 'About Yelp', 'Careers', 'Press', 'Investor Relations', 'Trust & Safety', 'Content Guidelin

In [33]:
browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166")
rest=browser.find_elements_by_class_name("arrange__09f24__LDfbsgutter-1__09f24__yAbCL")
p=1
for i in rest:
    print(i.text)

In [ ]:
review__09f24__oHr9V

In [84]:
browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166")
rest=browser.find_elements_by_class_name("review__09f24__oHr9V")
for i in rest:
    z = i.text.split("\n")
    print("Reviewer name: ",z[0])
    length = [len(p) for p in z]
    print("Reviewer's comment: ",z[length.index(max(length))])

['Mike L.', 'San Francisco, CA', '0', '8', '4', '9/6/2022', 'I have been coming to this store daily for over 5 years. Staff is very friendly and always greet me by my name. The coffee is very consistent in taste and quality. Hoping the management can keep the same staff for a long period of time.', 'Useful', 'Funny', 'Cool']
Reviewer name:  Mike L.
Reviewer's comment:  I have been coming to this store daily for over 5 years. Staff is very friendly and always greet me by my name. The coffee is very consistent in taste and quality. Hoping the management can keep the same staff for a long period of time.
['Colleen C.', 'Beverly Hills, CA', '7', '7', '11', '8/5/2022', '1 photo', "Had my coffee with Starbucks yesterday and I got to say I missed it! I missed the smell of the stores when you come in, the sound of the grinding and dripping coffee beans, and the greetings from the staff. I ordered their Café Latte with double shots and it help me kicked off my day. I'm thinking of having a coff

['Jay S.', 'Tampa, FL', '194', '44', '11', '12/5/2021', 'Food was overpriced and mediocre. Would not recommend. Service was ok. We went on a Friday night and it was fairly busy for this restaurant.', 'Useful', 'Funny 1', 'Cool']
Reviewer name:  Jay S.
Reviewer's comment:  Food was overpriced and mediocre. Would not recommend. Service was ok. We went on a Friday night and it was fairly busy for this restaurant.
['Cindy M.', 'Dublin, CA', '28', '14', '20', '9/21/2021', 'Customer service is horrendous. Additionally, my drink was made incorrectly in a grande size and when I asked to have it in a venti, all they did was add it into a venti cup instead of making a new order. Absolutely terrible.', 'Useful 1', 'Funny', 'Cool']
Reviewer name:  Cindy M.
Reviewer's comment:  Customer service is horrendous. Additionally, my drink was made incorrectly in a grande size and when I asked to have it in a venti, all they did was add it into a venti cup instead of making a new order. Absolutely terrible

In [17]:
n = int(9)
str((n-1)*10)

'80'

In [56]:
review_pages = 0
reviwer_name = []
reviewer_reviews = []
customer_name = []
Review_datasetPage0 = pd.DataFrame()

pages=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
existing_url = str(browser.current_url)
print(existing_url)

for i in pages:
    z = int(i.text.split('\n')[-1].split('of')[-1])
    print(z)


# browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166")
# # existing_url = str(browser.current_url)
# # page_num=2
# # print(existing_url+"?start="+str((page_num-1)*10))                 
# pages=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
# for i in pages:
#     review_pages = int(i.text.split('\n')[-1][-3:])
    

# for page_num in range(1,review_pages+1):
#     browser.get("https://www.yelp.com/biz/starbucks-san-francisco-166"+"?start="+str((page_num-1)*10))
#     reviews=browser.find_elements_by_class_name("review__09f24__oHr9V")
#     for i in reviews:
#         z = i.text.split("\n")
#         reviwer_name.append(z[0])
#         length = [len(p) for p in z]
#         reviewer_reviews.append(z[length.index(max(length))])
# Review_datasetPage0 = pd.DataFrame(list(zip(reviwer_name, reviewer_reviews)),columns =['Restaurant_Name','Reviewer_Name', 'Review'])
# Review_datasetPage0.head()

https://www.yelp.com/biz/2-brothers-bar-and-restaurant-decatur?osq=Restaurants
2


In [28]:
Review_datasetPage0.head()

,Reviewer_Name,Review
0,Mike L.,I have been coming to this store daily for ove...
1,Emam H.,I have experienced good at Starbucks. After t...
2,Colleen C.,Had my coffee with Starbucks yesterday and I g...
3,Joe J.,This video game is it's called stray and I die...
4,Kenneth W.,Keily provided the worst customer service I ha...


In [84]:
l1=['s','s','s','s','s']
l2=[1,2,3,4,5]
l3=['r','a','j','a','t']
Review_per_Rest = pd.DataFrame(list(zip(l1,l2, l3)),columns =['Restaurant_Name','Reviewer_Name', 'Review'])
Review_per_Rest

,Restaurant_Name,Reviewer_Name,Review
0,s,1,r
1,s,2,a
2,s,3,j
3,s,4,a
4,s,5,t


In [90]:
a = ['s']
b = a*5
print(b)

['s', 's', 's', 's', 's']


In [56]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)
GlobalData=pd.DataFrame()
counter=0
master_df = pd.DataFrame()
for zipcode in [30501]:
    print("ZipCode:- ",zipcode)
    print("Counter:- ",counter)
    browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
    list1=[]
    for i in browser.find_elements_by_class_name("css-chan6m"):
        list1.append(i.text)
    noOfPages=int(list1[-4][-2:])
    customerList=[]
    SubMenu=[]
    menuDesc=[]
    Menu=[]
    custDict={}
    FinalList=[]
    restType=[]
    RestaurantType=[]
    LocalData=pd.DataFrame()
    LocalDataPage0=pd.DataFrame()
    reviwer_name = []
    reviewer_reviews = []
    print("No of Pages:",noOfPages)
    Review_dataset = pd.DataFrame()
    Review_datasetPage0 = pd.DataFrame()
    Review_dataset_else_page = pd.DataFrame()
    Review_Dataset_2 = pd.DataFrame()
    combined_df = pd.DataFrame() 
    
    print("********************************************")
    for page in range(noOfPages):
        if(page==0):
#             print("--------------------------------------------Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
                print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])
                    
                ########## Extract Reviews ###############
                review_pages = 0
                reviwer_name = []
                reviewer_reviews = []
                pages=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
                for i in pages:
                    review_pages = int(i.text.split('\n')[-1].split('of')[-1])

                for page_num in range(1,review_pages+1):
                    existing_url = str(browser.current_url)
                    browser.get(existing_url+"?start="+str((page_num-1)*10))
                    reviews=browser.find_elements_by_class_name("review__09f24__oHr9V")
                    for i in reviews:
                        z = i.text.split("\n")
                        reviwer_name.append(z[0])
                        length = [len(p) for p in z]
                        reviewer_reviews.append(z[length.index(max(length))])
                        
#                 print(customerName)
#                 print(len(reviwer_name))
#                 print(reviwer_name)
#                 print(len(reviewer_reviews))
#                 print(reviewer_reviews)
                restaurant_name = [customerName]*len(reviwer_name)
                Zipcode = [zipcode]*len(reviwer_name)
                Review_per_Rest = pd.DataFrame(list(zip(Zipcode,restaurant_name,reviwer_name,reviewer_reviews)),columns =['Zipcode','Restaurant_Name','Reviewer_Name', 'Review'])
#                 print("This rest df: ",Review_per_Rest.shape)
                Review_datasetPage0 = pd.concat([Review_datasetPage0,Review_per_Rest],axis=0)
                print("This is 1st loop dataset page0: ",Review_datasetPage0.shape)
#                 print("First page Dataset accumulation: ",Review_datasetPage0.shape)
                    ##########################################
                
                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))
                        
                    data.loc[0,"RestaurantType"]=RestaurantType
                    browser.find_elements_by_class_name("css-qyp8bo")
                    
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])

                    
    
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
                
                    
                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
                    
            
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
                    
                    Menu["Key"]=1
    
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])

                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))
        
        else:
#             print("--------------------------------------------Else Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+str("0"*(5-len(str(zipcode)))+str(zipcode))+"&start="+str(page*10))
            browser.switch_to.window(browser.window_handles[0])
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
                print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])
                
                ########## Extract Reviews ###############
                review_pages = 0
                reviwer_name = []
                reviewer_reviews = []
                pages=browser.find_elements_by_class_name("pagination__09f24__VRjN4")
                for i in pages:
                    review_pages = int(i.text.split('\n')[-1].split('of')[-1])

                for page_num in range(1,review_pages+1):
                    existing_url = str(browser.current_url)
                    browser.get(existing_url+"?start="+str((page_num-1)*10))
                    reviews=browser.find_elements_by_class_name("review__09f24__oHr9V")
                    for i in reviews:
                        z = i.text.split("\n")
                        reviwer_name.append(z[0])
                        length = [len(p) for p in z]
                        reviewer_reviews.append(z[length.index(max(length))])
                        
                restaurant_name = [customerName]*len(reviwer_name)
                Zipcode = [zipcode]*len(reviwer_name)
                print("Else Page Restaurants: ",customerName)
                Review_per_Rest_inelse = pd.DataFrame(list(zip(Zipcode,restaurant_name,reviwer_name,reviewer_reviews)),columns =['Zipcode','Restaurant_Name','Reviewer_Name', 'Review'])
                Review_dataset_else_page = pd.concat([Review_dataset_else_page,Review_per_Rest_inelse],axis=0)
                print("This is 2nd loop dataset page_else: ",Review_dataset_else_page.shape)
                
                combined_df = pd.concat([Review_datasetPage0,Review_dataset_else_page],axis=0)
#                 print("This rest df: ",Review_per_Rest.shape)
#                 print("First page Dataset accumulation: ",Review_datasetPage0.shape) 

                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))
                    data.loc[0,"RestaurantType"]=RestaurantType

                    #################################################
                    browser.find_elements_by_class_name("css-qyp8bo")
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])
                
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
            
            

                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
    
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
#                     print("SubMenu",SubMenu)
                    Menu["Key"]=1
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])
                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))
                    LocalData=LocalData.dropna(subset=['ZipCode'])
                    

#     print("Append Zipcode",counter)
    
    GlobalData=GlobalData.append(LocalData, ignore_index=True)
    master_df = pd.concat([master_df,combined_df],axis=0)
    Review_Dataset_2 = pd.DataFrame()
    Review_Dataset_2 = pd.concat([Review_datasetPage0,Review_dataset_else_page],axis=0)
#     print("Review_dataset_2 shape: ",Review_dataset_2.shape)
        
#     if (counter %  5 ==0) & (counter!=0):
#         print("****Writing Data****",counter)
#         GlobalData.to_excel(r"CustomerAcquisition/customerAcquisition_till_Zipcode_"+zipcode+"_"+str(counter+1)+"_Zipcodes"+".xlsx")
#         GlobalData=pd.DataFrame()
#     counter+=1

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\rkumarrakeshkumar\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


ZipCode:-  30501
Counter:-  0
No of Pages: 24
********************************************
i: 1. Kelley’s Tavern & Grill
This is 1st loop dataset page0:  (30, 4)
i: 2. Standard Service
This is 1st loop dataset page0:  (60, 4)
i: 3. Atlanta Highway Seafood Market
This is 1st loop dataset page0:  (440, 4)
i: 4. Hopscotch Restaurant
This is 1st loop dataset page0:  (460, 4)
i: 5. The Inked Pig
This is 1st loop dataset page0:  (560, 4)
i: 6. Avocados Restaurant
This is 1st loop dataset page0:  (750, 4)
i: 7. Rabbittown Cafe
This is 1st loop dataset page0:  (860, 4)
i: 8. Antebellum
This is 1st loop dataset page0:  (1070, 4)
i: 9. Sweet Magnolias Cafe & Bakery
This is 1st loop dataset page0:  (1150, 4)
i: 10. Luna’s Restaurant
This is 1st loop dataset page0:  (1250, 4)
i: 11. YellowFin
Else Page Restaurants:  11. YellowFin
This is 2nd loop dataset page_else:  (90, 4)
i: 12. Zen Ramen & Sushi Burrito
Else Page Restaurants:  12. Zen Ramen & Sushi Burrito
This is 2nd loop dataset page_else:  (

i: 72. Haydees Cafe
Else Page Restaurants:  72. Haydees Cafe
This is 2nd loop dataset page_else:  (5826, 4)
i: 73. Vinny’s Little Italy
Else Page Restaurants:  73. Vinny’s Little Italy
This is 2nd loop dataset page_else:  (5846, 4)
i: 74. The Twisted Oar
Else Page Restaurants:  74. The Twisted Oar
This is 2nd loop dataset page_else:  (5956, 4)
i: 75. Wild Wing Cafe
Else Page Restaurants:  75. Wild Wing Cafe
This is 2nd loop dataset page_else:  (6066, 4)
i: 76. Delizio
Else Page Restaurants:  76. Delizio
This is 2nd loop dataset page_else:  (6096, 4)
i: 77. The Bakers Daughter
Else Page Restaurants:  77. The Bakers Daughter
This is 2nd loop dataset page_else:  (6097, 4)
i: 78. Mei Garden
Else Page Restaurants:  78. Mei Garden
This is 2nd loop dataset page_else:  (6147, 4)
i: 79. Manhattan Chicago Pizzeria
Else Page Restaurants:  79. Manhattan Chicago Pizzeria
This is 2nd loop dataset page_else:  (6167, 4)
i: 80. Hibachi Express
Else Page Restaurants:  80. Hibachi Express
This is 2nd loo

Else Page Restaurants:  140. Outback Steakhouse
This is 2nd loop dataset page_else:  (9051, 4)
i: 141. Inman Perk Coffee
Else Page Restaurants:  141. Inman Perk Coffee
This is 2nd loop dataset page_else:  (9141, 4)
i: 142. Shazzys Bar & Grill
Else Page Restaurants:  142. Shazzys Bar & Grill
This is 2nd loop dataset page_else:  (9171, 4)
i: 143. Supermercado Los 3 Hermanos
Else Page Restaurants:  143. Supermercado Los 3 Hermanos
This is 2nd loop dataset page_else:  (9176, 4)
i: 144. Taco Mac Gainesville
Else Page Restaurants:  144. Taco Mac Gainesville
This is 2nd loop dataset page_else:  (9176, 4)
i: 145. Shogun Japanese Steak and Sushi Bar
Else Page Restaurants:  145. Shogun Japanese Steak and Sushi Bar
This is 2nd loop dataset page_else:  (9256, 4)
i: 146. JAG’s Cafe and Catering
Else Page Restaurants:  146. JAG’s Cafe and Catering
This is 2nd loop dataset page_else:  (9257, 4)
i: 147. The Chattahoochee Grill
Else Page Restaurants:  147. The Chattahoochee Grill
This is 2nd loop datas

Else Page Restaurants:  206. El Caldero Dominican Restaurant
This is 2nd loop dataset page_else:  (10920, 4)
i: 207. Chipotle Mexican Grill
Else Page Restaurants:  207. Chipotle Mexican Grill
This is 2nd loop dataset page_else:  (10923, 4)
i: 208. American Deli
Else Page Restaurants:  208. American Deli
This is 2nd loop dataset page_else:  (10943, 4)
i: 209. La Esquina Latina Restaurant
Else Page Restaurants:  209. La Esquina Latina Restaurant
This is 2nd loop dataset page_else:  (10952, 4)
i: 210. Del Taco
Else Page Restaurants:  210. Del Taco
This is 2nd loop dataset page_else:  (10972, 4)
i: 211. Del Taco
Else Page Restaurants:  211. Del Taco
This is 2nd loop dataset page_else:  (10992, 4)
i: 212. Five Guys
Else Page Restaurants:  212. Five Guys
This is 2nd loop dataset page_else:  (11012, 4)
i: 213. Cook Out
Else Page Restaurants:  213. Cook Out
This is 2nd loop dataset page_else:  (11082, 4)
i: 214. Taqueria el amigo
Else Page Restaurants:  214. Taqueria el amigo
This is 2nd loop 

In [37]:
GlobalData.shape

(2025, 7)

In [39]:
GlobalData.head()

,Customer,ZipCode,RestaurantType,Menu,MenuDescription,Price,CityZipcode
0,2 Hudson Hil’s Market & Café,10516,"American (Traditional), Breakfast & Brunch",Farm Fresh Burger,Hudson valley grass fed beef with choice of ch...,$15.75,12550
1,2 Hudson Hil’s Market & Café,10516,"American (Traditional), Breakfast & Brunch",Quinoa and Black Bean Veggie Burger,"Veggie burger made with sweet potatoes, quinoa...",$13.75,12550
2,2 Hudson Hil’s Market & Café,10516,"American (Traditional), Breakfast & Brunch",Loaded Pork Green Chili Cheese Fries,2 reviews,$7.75,12550
3,2 Hudson Hil’s Market & Café,10516,"American (Traditional), Breakfast & Brunch",French Fries,2 reviews,$4.25,12550
4,2 Hudson Hil’s Market & Café,10516,"American (Traditional), Breakfast & Brunch",Salmon Salad,"Salmon cakes made from fresh salmon, dill and ...",$14.95,12550


In [41]:
GlobalData.Customer.nunique()

28

In [36]:
zip_12550 = combined_df.copy(deep=True)

In [45]:
zip_12550.to_csv(r'C:\Users\rkumarrakeshkumar\Desktop\AI_Guild_Capstone_Project\zip_12550.csv',index=False)

In [24]:
zip_46733_41501 = combined_df.copy(deep=True)

In [42]:
combined_df.shape

(25526, 4)

In [43]:
combined_df.head()

,Zipcode,Restaurant_Name,Reviewer_Name,Review
0,12550,1. Orange Hill Restaurant,Deborah H.,Sorry I didn't take pictures for this review. ...
1,12550,1. Orange Hill Restaurant,Jennifer L.Elite 2022,My kid loved the salmon and the other kid love...
2,12550,1. Orange Hill Restaurant,Andy C.Elite 2022,Found on yelp. Service was OK spotty.
3,12550,1. Orange Hill Restaurant,Nina C.Elite 2022,"I remember when this place used to be a diner,..."
4,12550,1. Orange Hill Restaurant,Patrick K.,This was our 3rd time here and it seems to get...


In [33]:
Review_dataset_else_page.shape

(4922, 4)

In [ ]:
zip_

In [19]:
zip_12550.shape

(25960, 4)

In [11]:
combined_df.Restaurant_Name.nunique()

166

In [10]:
combined_df.Zipcode.unique()

array([12550], dtype=int64)

In [110]:
Review_dataset_else_page.shape

(198, 3)

In [111]:
Review_datasetPage0.shape

(256, 3)

In [97]:
Review_datasetPage0.Restaurant_Name.unique()

array(['1. Back 40 Junction Restaurant', '2. West End Restaurant',
       '3. Soul Pig BBQ', '4. Corner Cafe', '5. Galley',
       '6. Indiana Pancake House', '7. Famous Monster Pizza', '8. Peking',
       '9. 2 Brothers Bar & Restaurant', '10. Fiesta Grill'], dtype=object)

In [98]:
Review_dataset_else_page.Restaurant_Name.unique()

array(['11. A1 Kitchen', '13. Hardee’s', '14. The Java Bean Cafe',
       '15. El Camino Real', '16. Chicote’s Mexican Grill',
       '17. Riverview Tap', '18. Richards Restaurant',
       '20. Small Town Bar & Grill', '21. Pizza King of Decatur',
       '23. Wings Etc', '24. Casey’s', '25. McDonald’s',
       '26. Monroe Drive Thru & Dining', '27. Taco Bell',
       '29. Flippers Riverside Tavern', '30. Wendy’s', '31. Burger King',
       '32. Dairy Queen Grill & Chill', '33. Papa Johns Pizza',
       '34. Pizza Hut', '36. Arby’s', '38. Subway - Temp. CLOSED'],
      dtype=object)

In [ ]:

#using random delay for time.sleep()
delays = [7, 4, 6, 2, 10, 19]
delay = np.random.choice(delays)

def get_review(url, res_name, res_address):
    binary = FirefoxBinary('/usr/bin/firefox')
    opts = webdriver.FirefoxOptions()
    opts.add_argument("--headless")
    driver = webdriver.Firefox(firefox_binary=binary, firefox_options=opts )
    driver.get(url)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    review_num = 0
    final_data = []
    num_page = 1
    info_scraped = {}
    info_scraped['reviewer_name'] = None
    #info_scraped['reviewer_stat'] = None
    info_scraped['reviewer_friends'] = None
    info_scraped['reviewer_reviews'] = None
    info_scraped['reviewer_photos'] = None
    info_scraped['ratings'] = None
    info_scraped['comment'] = None
    info_scraped['review_date'] = None
    info_scraped['reviewer_origin'] = None
    info_scraped['reviewer_profile'] = None
    # retrieve the total page number, if there is no information about this, it means the reviews have less than one full page, set page number to 1.
    try:
        total_page = driver.find_element_by_xpath('/html/body/div[2]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[2]/span').text
        print(total_page)
        totalpage = [int(s) for s in total_page.split() if s.isdigit()]

        num_page = totalpage[-1]
        print(num_page)
        #num_page = 1

    except:
        print(None)

    # iterate through all pages
    
    print(url)
    for page_np in range(num_page):
        print('[{}] {} scraped page out of {}'.format(datetime.now(), page_np, num_page))
        page = driver.page_source
        #soup = BeautifulSoup(page, 'lxml')
        soup2 = BeautifulSoup(page, 'lxml')
        
        # retrieve all data on the site
        all = soup.find_all('div', {'class': "main-content-wrap main-content-wrap--full"})
 
        #special_all_stat = soup2.find_all('div',{'class': " margin-t0-5__373c0__1VMSL border-color--default__373c0__3-ifU"})
        special_all_reviews = soup2.find_all('div',{'class': "review__373c0__13kpL border-color--default__373c0__3-ifU"})
        review_num += len(special_all_reviews)

        for i in range(len(special_all_reviews)):
            info_scraped = {}
            default = 'https://www.yelp.com'
            stat = ''
            origin = ''
            # retrieve reviewer name
            try:
                special_user = special_all_reviews[i].find('div',{'class': "user-passport-info border-color--default__373c0__3-ifU"})
                
                info_scraped['reviewer_name'] = special_user.find('a').text
                #print(info_scraped['reviewer_name'])
            except:
                print(None)

            # retrieve reviewer statistic, like number of friends, number of reviews, elite or not.
            try:
                
                for j in special_all_reviews[i].find_all('span', {'class': 'css-1dgkz3l'}) :
                    stat += j.text
                    stat += " "
                #print(stat)
                info_scraped['reviewer_friends'] = stat.split()[0]
                info_scraped['reviewer_reviews'] = stat.split()[1]
                info_scraped['reviewer_photos'] = stat.split()[2]
            except:

                print(None)

            # retrieve the rating of this review
            try:
                
                info_scraped['ratings'] = special_all_reviews[i].find('div', {"aria-label": re.compile('star rating')})["aria-label"].split()[0]
                #print(info_scraped['ratings'])
            except:
                print(None)

            # retrieve the comment text the reviewer left
            try:
                
                info_scraped['comment'] = special_all_reviews[i].find('p', {'class': 'comment__373c0__1M-px css-n6i4z7'}).find('span', {'class': 'raw__373c0__3rcx7'}).text
               
                #print(info_scraped['comment'])
            except:

                print(None)

            # retrieve the date of review
            try:
                info_scraped['review_date'] = datetime.strptime(special_all_reviews[i].find('span', {
                    'class': 'css-e81eai'}).text,'%m/%d/%Y').date()

                #print(info_scraped['review_date'])
            except:
                print(None)

            # retrieve origin of the reviewer and append them to one string
            try:
                origin = special_all_reviews[i].find('span',{'class':'css-n6i4z7'}).text
                 
                info_scraped['reviewer_origin'] = origin
                
            except:
                print(None)

            # retrieve profile website of each reviewer, prepared to retrieve the his history of ratting record
            try:
                info_scraped['reviewer_profile'] = default + special_all_reviews[i].find('a', {'class': 'css-166la90'}).attrs['href']
               
            except:
                print(None)
            print("********************************************************************************")
            
            final_data.append(info_scraped)
            
        #find no of clickable buttons
        clickable_button = soup.find_all('div',{'class': "pagination-link-container__373c0__1mmdE border-color--default__373c0__3-ifU"})
        clicking_links = len(clickable_button)+2
        click_link = str(clicking_links)
        
        # click the next button to go to next page
            
        if page_np == num_page-1:
            break
            
        else:
            driver.find_element_by_xpath(
                '//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[1]/div/div['+click_link+']/span/a/span').click()

    address = res_address.strip()
    restaurant_name = [res_name] * review_num
    address = [address] * review_num

    driver.quit()

    df = pd.DataFrame(final_data)

    df['restaurant name'] = pd.Series(restaurant_name)
    df['address'] = pd.Series(address)
    df.index += 1
    print(df)
    return df


iteration_from = 10
iteration_end = 20
review_data = []

for i in range(iteration_from, iteration_end):
    print(str(i) + " restaurant out of " + str(len(urls)))
    item = urls[i]
    name = res_name[i]
    address = res_add[i]
    resreview = get_review(item, name, address)
    review_data.append(resreview)
    review_all = pd.concat(review_data)
    review_all.to_csv("Reviews"+str(iteration_from)+"-"+str(iteration_end)+".csv")


In [95]:
GlobalData=pd.DataFrame()
counter=0
for zipcode in [46733]:
    print("ZipCode:- ",zipcode)
    print("Counter:- ",counter)
    browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
    list1=[]
    for i in browser.find_elements_by_class_name("css-chan6m"):
        list1.append(i.text)
    noOfPages=int(list1[-4][-2:])
    customerList=[]
    SubMenu=[]
    menuDesc=[]
    Menu=[]
    custDict={}
    FinalList=[]
    restType=[]
    RestaurantType=[]
    LocalData=pd.DataFrame()
    LocalDataPage0=pd.DataFrame()
    print("No of Pages:",noOfPages)
    print("********************************************")
    for page in range(noOfPages):
        if(page==0):
#             print("--------------------------------------------Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+("0"*(5-len(str(zipcode)))+str(zipcode)))
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
#                 print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])
                
                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))

                        
                    data.loc[0,"RestaurantType"]=RestaurantType
                    browser.find_elements_by_class_name("css-qyp8bo")
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])

                    
    
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
                
                    
                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
                    
            
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
                    
                    Menu["Key"]=1
    
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])

                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))

        else:
#             print("--------------------------------------------Else Page: ",page,"-----------------------------------------------------")
            browser.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc="+str("0"*(5-len(str(zipcode)))+str(zipcode))+"&start="+str(page*10))
            browser.switch_to.window(browser.window_handles[0])
            rest=browser.find_elements_by_class_name("css-1egxyvc")
            for i in rest:
#                 print("i:",i.text)
                i.click()
                customerName=i.text
                if(len(browser.window_handles)>1):
                    browser.switch_to.window(browser.window_handles[1])

                if(len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]"))==0):
#                     print("inIF: ",customerName,"length",len(browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")))
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                        browser.switch_to.window(browser.window_handles[0])
                        continue
                    else:
                        browser.back()
                    continue
                else:
#                     print("inElse: ")
#                     print(customerName)
                    data=pd.DataFrame()
                    data.loc[0,"Customer"]=customerName
                    city=[]
                    for i in browser.find_elements_by_class_name("css-qyp8bo"):
                        city.append(i.text)
                        for i in city:
                            if (len(i.split(","))>1):
                                Name=i[-5:]
                    data.loc[0,"ZipCode"]=Name
                    SubMenu=[]
                    menuDesc=[]
                    priceInfo=[]
                    RestaurantType=[]
                    restType=[]
                    Menu=pd.DataFrame()
                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                    Type=browser.find_elements_by_class_name("css-1fdy0l5")
                    for Typei in Type:
                        restType.append(Typei.text)
                    RestaurantType.append(" ".join(restType[1:-1]))
                    data.loc[0,"RestaurantType"]=RestaurantType
                    

                    browser.find_elements_by_class_name("css-qyp8bo")
                    browser.find_elements_by_xpath("//span[contains(text(),'Full menu')]")[0].click()
                    MenuAndDesc=browser.find_elements_by_class_name("menu-item")
                    for menu in range(len(MenuAndDesc)):
                        SubMenu.append(MenuAndDesc[menu].text.split("\n")[0])
                
                    for desc in range(len(MenuAndDesc)):
                        if(len(MenuAndDesc[desc].text.split("\n"))<=1):
                            menuDesc.append(" ")
                        else:
                            menuDesc.append(MenuAndDesc[desc].text.split("\n")[1])
                

                    for price in range(len(MenuAndDesc)):
                        priceInfo.append(MenuAndDesc[price].text.split("\n")[-1])
    
                    Menu["Menu"]=SubMenu
                    Menu["MenuDescription"]=menuDesc
                    Menu["Price"]=priceInfo
#                     print("SubMenu",SubMenu)
                    Menu["Key"]=1
                    data["Key"]=1
                    data = pd.merge(data, Menu, on ='Key').drop("Key", 1)

                    if(len(browser.window_handles)>1):
                        browser.switch_to.window(browser.window_handles[1])
                        browser.close()
                    browser.switch_to.window(browser.window_handles[0])
                    LocalData=LocalData.append(data, ignore_index=True)
                    LocalData["Customer"]=LocalData["Customer"].str.replace(r'[0-9][.]', '', regex=True).str.lstrip().str.rstrip()
                    LocalData["CityZipcode"]=str("0"*(5-len(str(zipcode)))+str(zipcode))
                    LocalData=LocalData.dropna(subset=['ZipCode'])
                    

#     print("Append Zipcode",counter)
    
    GlobalData=GlobalData.append(LocalData, ignore_index=True)
        
    if (counter %  5 ==0) & (counter!=0):
        print("****Writing Data****",counter)
        GlobalData.to_excel(r"CustomerAcquisition/customerAcquisition_till_Zipcode_"+zipcode+"_"+str(counter+1)+"_Zipcodes"+".xlsx")
        GlobalData=pd.DataFrame()
    counter+=1

ZipCode:-  46733
Counter:-  0
No of Pages: 4
********************************************
